<img src="https://github.com/GodStorm91/mobydock/blob/master/mobydock/static/docker-logo.png?raw=true" width="10%"> ➕
<img src="https://github.com/Microsoft/azuredatastudio/blob/master/samples/notebookSamples/Graphics/AzureDataStudioLogo.png?raw=true" width="6.5%"> = ❤

# This PowerShell Notebook will walk you through how to setup a SQL-on-Linux instance in a Docker container, on your local [Windows] machine.

## Preparation Steps
0A) Before any of this can work, you must have Docker Destop running. 
    You must also have the latest SqlServer module installed from the PowerShell Gallery.
Install-Module SqlServe

In [0]:
Import-Module SqlServer;

0C) Create a local directory for the container to use to store database files, outside of the container.

In [2]:
mkdir C:/SQLData/Docker/SQLDev03

mkdir : An item with the specified name C:\SQLData\Docker\SQLDev03 already exists.
At line:1 char:5
+ . { mkdir C:/SQLData/Docker/SQLDev03 }
+     ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ResourceExists: (C:\SQLData\Docker\SQLDev03:String) [New-Item], IOException
    + FullyQualifiedErrorId : DirectoryExist,Microsoft.PowerShell.Commands.NewItemCommand
 


## Create your SQL-on-Linux Docker Container
1) Create a SQL-on-Linux Docker Container with just the code below.
    NOTE: You may want to change the password used for the SA account.
    You may also want to change the Volumne path: "C:\SQLData\Docker\SQLDev03"

In [2]:
docker create -v /sqldata -v /sqllog --name sqlfiles03 ubuntu

Unable to find image 'ubuntu:latest' locally
latest: Pulling from library/ubuntu
7ddbc47eeb70: Pulling fs layer
c1bbdc448b72: Pulling fs layer
8c3b70e39044: Pulling fs layer
45d437916d57: Pulling fs layer
45d437916d57: Waiting
c1bbdc448b72: Verifying Checksum
c1bbdc448b72: Download complete
8c3b70e39044: Verifying Checksum
8c3b70e39044: Download complete
45d437916d57: Verifying Checksum
45d437916d57: Download complete
7ddbc47eeb70: Verifying Checksum
7ddbc47eeb70: Download complete
7ddbc47eeb70: Pull complete
c1bbdc448b72: Pull complete
8c3b70e39044: Pull complete
45d437916d57: Pull complete
Digest: sha256:6e9f67fa63b0323e9a1e587fd71c561ba48a034504fb804fd26fd8800039835d
Status: Downloaded newer image for ubuntu:latest
1054d98e3fe0f4063dbcb6eafd6aae20f77bf7fd69cc07a2336fa6fcb09ac947


In [4]:
docker run -d -p 10003:1433 -v C:\SQLData\Docker\SQLDev03:/sqlserver -e ACCEPT_EULA=Y -e SA_PASSWORD=Test1234 --name testcontainer03 mcr.microsoft.com/mssql/server:2019-latest

41f9a28848ab7a79bd273705ced22747c1ad88d38945b96821c7dc74fe04260e


Verify that the SQL instance in the container is up & running

In [6]:
$SqlCred = (Get-Credential sa)
Get-SqlDatabase -ServerInstance 'localhost,10003' -Credential $SqlCred


Name                 Status           Size     Space  Recovery Compat. Owner                     Collation             
                                            Available  Model     Level                                                 
----                 ------           ---- ---------- -------- ------- -----                     ---------             
master               Normal        6.44 MB 1008.00 KB Simple       150 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
model                Normal       16.00 MB    5.70 MB Full         150 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
msdb                 Normal       13.88 MB    1.41 MB Simple       150 sa                        SQL_Latin1_General_CP1
                                       

2) Copy the backup file to the directory your container volume is mapped to.
    Make sure you use the location you stored the .bak file in.

In [9]:
Copy-Item -Path "$($Home)\Downloads\AdventureWorks2016.bak" -Destination C:\SQLData\Docker\SQLDev03

3) Restore the AdventureWorks2016 databas

In [12]:
Restore-SqlDatabase -ServerInstance 'localhost,10003' -Credential $SqlCred -BackupFile '/sqlserver/AdventureWorks2016.bak' -Database 'AdventureWorks2016' -AutoRelocateFile

In [14]:
Get-SqlDatabase -ServerInstance 'localhost,10003' -Credential $SqlCred


Name                 Status           Size     Space  Recovery Compat. Owner                     Collation             
                                            Available  Model     Level                                                 
----                 ------           ---- ---------- -------- ------- -----                     ---------             
AdventureWorks2016   Normal      209.63 MB    1.31 MB Simple       130 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
master               Normal        6.44 MB 1008.00 KB Simple       150 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
model                Normal       16.00 MB    5.70 MB Full         150 sa                        SQL_Latin1_General_CP1
                                       